In [1]:
import os
import pickle
from tweet import Reply, PostNew

In [2]:
ROOT_PICKLE = '../pickle/'

In [3]:
lexicon_attack = ['falso', 'Falso', 'FALSO']
lexicon_support = ['verdad']

**Primer resultado**

In [4]:
# Corregir tweets del post, estoy guardando todos como si fuesen respuestas del tweet, y no lo son
# hay tweets que son respuestas de las respuestas

In [5]:
with open(os.path.join(ROOT_PICKLE, 'chequeado1186118160287698944.json.pkl'), 'rb') as fd:
    json = pickle.load(fd)

In [6]:
def preprocessing(json):
    replies = []
    post = None
    for id_str, tweet in json.items():
        
        full_text = tweet['full_text']
        reply_count = tweet['reply_count']
        created_at = tweet['created_at']
        display_text_range = tweet['display_text_range']
        conversation_id_str = tweet['conversation_id_str']
        
        if id_str == conversation_id_str:
            post = PostNew(id_str=id_str,
                           created_at=created_at,
                           reply_count=reply_count,
                           display_text_range=display_text_range,
                           conversation_id_str=conversation_id_str,
                           full_text=full_text
                          )
        else:
            in_reply_to_status_id_str = tweet['in_reply_to_status_id_str']
            #screen_name = tweet['screen_name']
            twt = Reply(created_at=created_at,
                        full_text=full_text,
                        id_str=id_str,
                        reply_count=reply_count,
                        #screen_name=screen_name,
                        display_text_range=display_text_range,
                        conversation_id_str=conversation_id_str,
                        in_reply_to_status_id_str=in_reply_to_status_id_str
                       )
            replies.append(twt)
    post.replies = replies
    return post

In [7]:
post = preprocessing(json)

In [9]:
post.id_str

'1186118160287698944'

In [12]:
post.replies[0].tokens

['Pero',
 'eso',
 'tiene',
 'algo',
 'que',
 'ver',
 'con',
 'que',
 'aumentó',
 'el',
 'número',
 'de',
 'familias',
 'pobres',
 '?',
 'Cada',
 'vez',
 'hay',
 'menos',
 'donde',
 'secuestrar',
 '.',
 'No',
 '?']

In [15]:
post.is_controversial(lexicon_attack)

True

In [39]:
threads = post.get_threads()

In [16]:
# Primera iteracion del pipeline
[tweet.full_text for tweet in post.replies_attack()]

['Tengo números que dicen lo contrario. Bajo. Bajo secuestros extorsivos. Pero no desaparecieron. Creo que Uds convenientemente usan el #EsVerdaderoPero. Sin embargo para mi aca es falso',
 'Los de “CHEQUEADO” son independientes.... FALSO',
 'cómo puede acreditar como verdadero un enunciado que contiene "prácticamente"? con este rigor dsp le ponen falso y exagerado a los demás',
 'Al usar en la frase el "prácticamente" creo que anula el verdadero y el falso.']

**Segundo resultado**

In [10]:
# See https://radimrehurek.com/gensim/models/word2vec.html
from gensim.models import KeyedVectors

In [7]:
model = KeyedVectors.load_word2vec_format("../model/SBW-vectors-300-min5.bin.gz", binary=True)

In [47]:
lexicon_attack += [word for word, d in model.most_similar(positive=["falso", "mentira"])]

In [48]:
lexicon_attack

['falso',
 'Falso',
 'FALSO',
 'falsa',
 'mentirilla',
 'verdad',
 'fraudulencia',
 'engaño',
 'infundio',
 'mentiras',
 'andrómina',
 'patraña',
 'encubriéndola']

In [35]:
# Segunda iteracion del pipeline

In [49]:
post.is_controversial(lexicon_attack)

True

In [50]:
[tweet.full_text for tweet in post.replies_attack()]

['Tengo números que dicen lo contrario. Bajo. Bajo secuestros extorsivos. Pero no desaparecieron. Creo que Uds convenientemente usan el #EsVerdaderoPero. Sin embargo para mi aca es falso',
 'Los de “CHEQUEADO” son independientes.... FALSO',
 'cómo puede acreditar como verdadero un enunciado que contiene "prácticamente"? con este rigor dsp le ponen falso y exagerado a los demás',
 'No. El secuestro virtual efectivamente es una estafa (nadie fue secuestrado, ahí está el ardid o engaño, y te llaman pidiendo un rescate).',
 'Al usar en la frase el "prácticamente" creo que anula el verdadero y el falso.']

**Tercer resultado**

flask

**API TWITTER**

In [25]:
import tweepy

In [26]:
CONSUMER_KEY = 'tRe3ZBhnXDPBEKyTi4ifvPTC7' 
CONSUMER_SECRET = '0dmQlSimyEmaOPiDoFZTF0wYjO0LSzSdWmCP7afZkNHZF94Af5'
ACCESS_TOKEN = '421837149-YVewRWLSu8xyYNU1H4NtELUejGB8lzvT4KCfallG'
ACCESS_SECRET = 'mFsKYpvJaWvZGHRoy874Z4IF09CRP69srlVj2yjN6mH5L'

In [27]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [32]:
search = tweepy.Cursor(api.search, q="to:FedeFavre", sinceId = 1188249861721919489).items()